In [ ]:
import sys
import os
import time
from easydict import EasyDict as edict

import cv2
import torch
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt

sys.path.append('../')

import config.astyx_config as cnf
from models.model_utils import create_model
from evaluate import evaluate_mAP

sys.path.append('/media/berens/T7/Dissertation/ComplexYOLO/Astyx/src/')

from data_process_astyx.astyx_dataloader import create_val_dataloader


In [ ]:
root = "/media/berens/T7/Dissertation/ComplexYOLO/Astyx/src/checkpoints/"

c = plt.colormaps()
colors = plt.cm.jet(np.linspace(0, 1, 6))

configs = edict({"arch": "darknet",
                 "cfgfile": "./config/cfg/complex_yolov4.cfg",
                 "use_giou_loss": False,
                 "device": "cpu",
                 "dataset_dir": f"{root}/../../dataset_astyx_hires2019",
                 "num_samples": 100,
                 "dist_prop_lidar": 1,
                 "dist_prop_radar": 1,
                 "disturb_types_training_lidar" : [None],
                 "disturb_levels_training_lidar": [0],
                 "disturb_types_training_radar" : [None],
                 "disturb_levels_training_radar": [0],
                 "set_seed": 1000,
                 "distributed": False,
                 "batch_size": 1,
                 "pin_memory": True,
                 "num_workers": 1,
                 "img_size": 608,
                 "conf_thresh": 0.5,
                 "nms_thresh": 0.5,
                 "iou_thresh": 0.5,
                })

In [ ]:
def create_config(configs, lidar_disturb, radar_disturb, seed):
    """
    Creates a new configuration based on the original configuration.
    
    Parameters:
    - configs: edict
        The original configuration to be copied.
    - lidar_disturb: str or None
        The type of disturbance for the LiDAR sensor.
    - radar_disturb: str or None
        The type of disturbance for the RADAR sensor.
    - seed: int
        The random seed to be used for the new configuration.
    
    Returns:
    - config_copy: edict
        The created configuration with specified disturbance types and the set seed.
    """
    config_copy = edict(configs.copy())
    config_copy.disturb_types_training_lidar = [lidar_disturb] if lidar_disturb else ['None']
    config_copy.disturb_types_training_radar = [radar_disturb] if radar_disturb else ['None']
    config_copy.set_seed = seed
    config_copy.subdivisions = int(64 / config_copy.batch_size)
    return config_copy

def evaluate_disturbance(config, level, model):
    """
    Evaluates the model's performance under the specified disturbance conditions.
    
    Parameters:
    - config: edict
        The configuration containing the current disturbance conditions.
    - level: float
        The level of disturbance to be applied during evaluation.
    - model: Model
        The model being evaluated.
    
    Returns:
    - precision[0]: float
        The precision of the model on the first class.
    - recall[0]: float
        The recall of the model on the first class.
    - AP[0]: float
        The average precision (AP) of the model on the first class.
    - f1[0]: float
        The F1 score of the model on the first class.
    """
    if config.disturb_types_training_lidar != ['None']:
        config.disturb_levels_training_lidar = [level]
    if config.disturb_types_training_radar != ['None']:
        config.disturb_levels_training_radar = [level]
    val_dataloader = create_val_dataloader(config)
    precision, recall, AP, f1, ap_class = evaluate_mAP(val_dataloader, model, config, None)
    return precision[0], recall[0], AP[0], f1[0]

def run_evaluations(config, model, levels):
    """
    Iterates through different disturbance levels and evaluates model performance for each.
    
    Parameters:
    - config: edict
        The configuration containing the current disturbance conditions.
    - model: Model
        The model being evaluated.
    - levels: list of floats
        The different levels of disturbance to be tested.
    
    Returns:
    - eval_values: np.ndarray
        An array of evaluation results across the different disturbance levels.
    """
    eval_values = np.zeros(len(levels))
    for level_id, level in enumerate(levels):
        eval_values[level_id] = evaluate_disturbance(config, level, model)
    return eval_values

def make_plot(eval_data, subplot, xticks = np.arange(0,1.1,0.1), title):
    """
    Plots evaluation data on the provided subplot with markers and colors corresponding to different levels of disturbance.

    Parameters:
    - eval_data: np.ndarray
        The evaluation data to plot, with rows corresponding to different proportions and columns to different disturbance levels.
    - subplot: matplotlib.axes.Axes
        The subplot on which the data will be plotted.
    - xticks: np.ndarray, optional
        The values to be used for the x-axis ticks, default is np.arange(0, 1.1, 0.1).
    - title: str, optional
        The title of the subplot, default is an empty string.
    
    Returns:
    - None
    """
    for eval_level in range(data_disturbed.shape[1]):
        subplot.plot(xticks,data_disturbed[:,eval_level],marker=".",markersize=10,c=colors[eval_level])

    subplot.plot(xticks,data_disturbed.mean(1),marker=".",markersize=10,c=[0.5, 0.5 , 0.5])
        
    subplot.set_xlabel("Proportion of introduced distortions",fontsize="x-large")
    subplot.set_ylim((0,1))
    subplot.set_ylabel(performanzmass,fontsize="x-large")
    subplot.set_xticks(np.arange(0,1.1,0.1))
    subplot.legend([plt.Line2D([], [], linestyle='-', marker='o', color=colors[0]),
                   plt.Line2D([], [], linestyle='-', marker='o', color=colors[1]),
                   plt.Line2D([], [], linestyle='-', marker='o', color=colors[2]),
                   plt.Line2D([], [], linestyle='-', marker='o', color=colors[3]),
                   plt.Line2D([], [], linestyle='-', marker='o', color=colors[4]),
                   plt.Line2D([], [], linestyle='-', marker='o', color=[0.5, 0.5 , 0.5 ])],
                   [0,1,2,3,4, "Mean"], title="Diff. Eval.",fontsize="x-large", title_fontsize="x-large")
    
    subplot.set_title(title,fontsize="x-large")
            
def multifactorial_performance(configs, pretrained_path_list, proportions, disturbances, levels, title_lidar, title_radar, title_both):
    """
    Evaluates model performance across multiple pretrained models, disturbances, and levels, and generates plots for each sensor type.

    Parameters:
    - configs: edict
        The configuration object containing settings for the model and evaluation.
    - pretrained_path_list: list of str
        List of file paths to the pretrained model checkpoints.
    - proportions: list of float
        List of proportions for introduced distortions, used for x-axis in the plot.
    - disturbances: list of str
        List of disturbance types to evaluate.
    - levels: list of float
        List of levels of disturbances.
    - title_lidar: str
        Title for the LiDAR evaluation plot.
    - title_radar: str
        Title for the RADAR evaluation plot.
    - title_both: str
        Title for the combined LiDAR and RADAR evaluation plot.

    Returns:
    - fig: matplotlib.figure.Figure
        The created Matplotlib figure with the results.
    - ax: numpy.ndarray of matplotlib.axes.Axes
        The axes of the subplots in the figure.
    - eval_values_lidar: np.ndarray
        The evaluation results for LiDAR disturbances.
    - eval_values_radar: np.ndarray
        The evaluation results for RADAR disturbances.
    - eval_values_both: np.ndarray
        The evaluation results for disturbances on both sensors.
    """
    seed = np.random.randint(10000)
    
    if configs.disturb_types_training_lidar != [None] and configs.disturb_types_training_radar != [None]:
        Sensor_T = "both"
    elif configs.disturb_types_training_lidar != [None]:
        Sensor_T = "LiDAR"
    elif configs.disturb_types_training_radar != [None]:
        Sensor_T = "RADAR"
    
    Type_T = configs.disturb_types_training_lidar
    
    eval_values_lidar = np.zeros(len(pretrained_path_list), len(disturbances), len(levels))
    eval_values_radar = np.zeros(len(pretrained_path_list), len(disturbances), len(levels))
    eval_values_both = np.zeros(len(pretrained_path_list), len(disturbances), len(levels))
    
    for pretrained_path_idx, pretrained_path in enumerate(pretrained_path_list):
        model = create_model(configs)
        model.load_state_dict(torch.load(pretrained_path,
                                         map_location=torch.device(configs.device)))
        model = model.to(device=configs.device)
        model = model.eval()
        
        for disturb_id, disturb in enumerate(disturbances):
            lidar_config = create_config(configs, disturb, None, seed)
            radar_config = create_config(configs, None, disturb, seed)
            both_config = create_config(configs, disturb, disturb, seed)

            eval_values_lidar[pretrained_path_idx, disturb_id] = run_evaluations(lidar_config, model)
            eval_values_radar[pretrained_path_idx, disturb_id] = run_evaluations(radar_config, model)
            eval_values_both[pretrained_path_idx, disturb_id] = run_evaluations(both_config, model)

    eval_values_lidar = eval_values_lidar.mean(axis = 1)
    eval_values_radar = eval_values_radar.mean(axis = 1)
    eval_values_both = eval_values_both.mean(axis = 1)
    
    fig, ax = plt.subplots(1,3,frameon=False)
    make_plot(eval_values_lidar, subplot = ax, xticks = proportions,
                 title_lidar)
    make_plot(eval_values_radar, subplot = ax, xticks = proportions,
                 title_radar)
    make_plot(eval_values_both, subplot = ax, xticks = proportions,
                 title_both)
    return fig, ax, eval_values_lidar, eval_values_radar, eval_values_both
    
    

# Low Level Fusion

In [ ]:
configs.low_fusion = True
configs.high_fusion = False
configs.lidar = False
configs.radar = False
configs.VR = False

disturbances = ['random_noise', 'random_loss', 'random_shift', 'blobs', 'intensity']
levels = [[0.], [0.25], [0.5], [0.75], [1.]]

pretrained_path_list = ["./models/complexyolov4_astyx_lidar/random_noise_4/Model_0_0.pht",
                       "./models/complexyolov4_astyx_lidar/random_noise_4/Model_0_1.pht",
                       "./models/complexyolov4_astyx_lidar/random_noise_4/Model_0_2.pht",
                       "./models/complexyolov4_astyx_lidar/random_noise_4/Model_0_3.pht",
                       "./models/complexyolov4_astyx_lidar/random_noise_4/Model_0_4.pht",
                       "./models/complexyolov4_astyx_lidar/random_noise_4/Model_0_5.pht",
                       "./models/complexyolov4_astyx_lidar/random_noise_4/Model_0_6.pht",
                       "./models/complexyolov4_astyx_lidar/random_noise_4/Model_0_7.pht",
                       "./models/complexyolov4_astyx_lidar/random_noise_4/Model_0_8.pht",
                       "./models/complexyolov4_astyx_lidar/random_noise_4/Model_0_9.pht",
                       "./models/complexyolov4_astyx_lidar/random_noise_4/Model_1_0.pht"]

In [ ]:
multifactorial_performance(configs, pretrained_path_list,
                           proportions, disturbances[0], levels,
                           title_lidar = "Complex-YOLO Low Level Training: Level = 4,\n Sensor = both, Disturbance = Random Noise \
\n Eval.: Sensor = LiDAR, Disturbance = Random Noise",
                           title_radar = "Complex-YOLO Low Level Training: Level = 4,\n Sensor = both, Disturbance = Random Noise \
\n Eval.: Sensor = RADAR, Disturbance = Random Noise",
                           title_both = "Complex-YOLO Low Level Training: Level = 4,\n Sensor = both, Disturbance = Random Noise \
\n Eval.: Sensor = both, Disturbance = Random Noise"):
